# Intrinsic Curvature Computations 

$
\newcommand{\bb}[1]{\mathbb{{#1}}}
\newcommand{\cl}[1]{\mathcal{{#1}}}
\newcommand{\PS}{P_{{\cl{S}}}}
\newcommand{\gradS}{\nabla_{\cl{S}}}
$

We have seen in [the previous notebook](Extrinsic.ipynb)  that Gauss curvature of a surface can be computed using the derivative of the normal vector of the surface.  Ever since the Gauss' Theorema Egregium (1827), we also know an alternate, miraculous, *intrinsic* way to compute the Gauss curvature of a surface. By  an "intrinsic" way, we mean what a being who lives within the surface is capable of; we mean that there is no need to use any outside information on the ambient space in which the surface may sit. In particular, we will have no need for the surface normal; one only needs a  metric tensor $g$ used to determine lengths and angles from within the surface.

In [ ]:
from netgen.occ import X, Y, Z, Circle, Pnt, Sphere, Box, Cylinder, Ellipsoid, Axes, Glue, OCCGeometry
from netgen import meshing
from ngsolve import Mesh, Cof, Grad, Integrate, Trace, Cross, Det, OuterProduct, InnerProduct, Normalize
from ngsolve import x, y, z, tan, atan2, cos, acos, sin, dx, ds, log, sqrt, pi
from ngsolve import CF, H1, VectorFacetSurface, HDivDivSurface, LinearForm, BilinearForm, GridFunction
from ngsolve.meshes import MakeStructured2DMesh
import ngsolve as ng
from ngsolve.webgui import Draw
MESHSURF = meshing.MeshingStep.MESHSURFACE

## Gauss curvature of smooth 2-manifolds 

From we have seen in [the previous notebook](Extrinsic.ipynb),  we know that Gauss curvature of a surface can be computed easily using the Weingarten tensor or the shape operator:

In [ ]:
def GaussCurvature():
    nu = ng.specialcf.normal(3)
    W = Grad(nu)
    return Det(W + OuterProduct(nu, nu))

This is the *extrinsic* approach for computing the Gauss curvature. It requires the *normal* vector $\nu$ of the surface, a vector that does not make sense without the ambient space into which the manifold is embedded.  

**Hemisphere example** 

As an example, consider a hemisperical surface of radius $R$. Its Gauss curvature must be $1/R^2$, the same as the Gauss curvature of a sphere of radius $R$. This agrees with the result computed by the extrinsic numerical computation below.

In [ ]:
from netgen.occ import HalfSpace, Vec
s = Sphere(Pnt(0,0,0), r=1)
h = HalfSpace(Pnt(0,0,0), Vec(0, 0, -1))
hemisphere = OCCGeometry((s*h).faces[0], dim=2)
hemisph_mesh = Mesh(hemisphere.GenerateMesh(maxh=0.4, perfstepsend=MESHSURF)).Curve(4)

Draw(GaussCurvature(), hemisph_mesh);

We will revisit this example multiple times below.

## The metric tensor

Surfaces embedded in $\bb R^3$ obtains its metric tensor $g$ from the ambient Euclidean domain and its embedding. In contrast, a general Riemannian 2-manifold comes with a metric tensor $g$ but has no embedding information.

To obtain the metric tensor $g$ of an embedded surface, it is typical to employ a parameter domain and a parameterization or an embedding as shown:

<img src="./embedding.png" width="40%">

Here variables $x$ and $y$ lie in the flat paramater domain  and the embedding $\theta(x, y)$  gives the points on the surface $\cl S$. The metric $g$ to measure lengths and angles on an embedded surface is then  given by 

$$
g = (\nabla \theta)^t (\nabla \theta)
$$

where $\nabla \theta$ is the $3\times 2$ matrix whose columns are derivatives of $\theta$ with respect to the two parameter variables. Clearly $g$ is a symmetric positive definite matrix field on the parameter domain.

**The hemisphere example**

As an example, consider the hemisphere again. A convenient paramater domain is the disk underneath the hemisphere.

In [ ]:
# x, y in a flat parameter domain, a unit disk: 
order  = 4
param_mesh = Mesh(OCCGeometry(Circle(Pnt(0,0), r=1).Face(), dim=2).GenerateMesh(maxh=0.2)).Curve(order)
Draw(param_mesh);

One can obtain a parameterization for this example by   thinking in spherical coordinates and then writing  the polar angle in terms of the parameter variables in the flat domain.

In [ ]:
# embedding theta(x, y) = (x, y, 0) + u(x,y) for a displacement vector u set below

r = sqrt(x**2+y**2)
phi = atan2(y,x)
u = GridFunction(H1(param_mesh,order=order)**3)
u.Interpolate( (cos(phi)*sin(r*pi/2) - x,
                sin(phi)*sin(r*pi/2) - y,
                cos(r*pi/2)) )

In [ ]:
# display embedding by ngsolve deformation of parameter domain

uxy = GridFunction(H1(param_mesh,order=order)**2)
uxy.Set ( u[0:2] )
param_mesh.SetDeformation(uxy)
Draw (u[2], param_mesh, deformation=True)
param_mesh.UnsetDeformation()

Since 

$$
\theta(x, y) = (x, y, 0) + u(x, y)
$$

its gradient  is given by 
$
\nabla \theta  = [e_1, e_2] + [\partial_x u, \partial_y u].
$
The metric $g = (\nabla \theta)^t (\nabla \theta)$ is computed from this gradient below. It is then  interpolated  into the Regge finite element space suitable for representing metrics.

In [ ]:
F = CF( (1,0,       # F = grad \theta
         0,1, 
         0,0), dims=(3,2)) + Grad(u)

g = F.trans*F

Reg = ng.HCurlCurl(param_mesh, order=3) 
gh = GridFunction(Reg)
gh.Set(g, dual=True, bonus_intorder=4)
Draw (gh[0, 0], param_mesh);

Now that we have the metric $g$ for this example, the next time we revisit this example, we will forget about where it came from (the embedding $\theta$) and deliberatly use only the metric  $g$ (or rather the the `gh` computed above) to illustrate how the intrinsic computation works.

## The intrinsic computation

In the intrinsic way to compute the Gauss curvature $K$,  One first defines the Christoffel symbols 

\begin{align*}
\Gamma_{ij}^k(g) 
	=\frac{1}{2}g ^{kl}\left(\partial_{i}g_{jl}+\partial_{j}g_{il}-\partial_{l}g_{ij}\right)=g ^{kl}\Gamma_{ijl}(g).
\end{align*}

using the summation convention (and accompanying conventions of raising indices
whereby $g^{kl}$ is the $(k,l)$-entry of the inverse of $g$). Then put 

\begin{align*}
R_{ijkl} = 
			\partial_i \Gamma_{jkl} - \partial_j \Gamma_{ikl}
			+ \Gamma_{ik}^p\Gamma_{jpl}
			- \Gamma_{jk}^p\Gamma_{ipl}
\end{align*}

for any indices $i,j,k,l$ taking values of either 0 or 1 for a 2-manifold.
The theorem then gives Gauss curvature $K$ as

\begin{align*}
K =\frac{R_{0110}}{\det g}.
\end{align*}

Clearly, this computation only requires knowledge of the metric $g$.


We can certainly compute the many derivatives above and end up with $K$. But `ngsolve` has already implemented these formulas to facilitate intrinsic curvature computation.  The $R_{0110}$ above represents a component of the general fourth-order Riemann tensor (which only has one independent component in two dimensions). It can computed using the method `Operator("Riemann")` of a `GridFunction` representing the  metric $g$.  


**The hemisphere example**

Reconsidering the same hemisphere example, for which we have computed a metric approximation `gh` previously, all that is needed to obtain a Gauss curvature approximation are the following lines:

In [ ]:
R = gh.Operator("Riemann")
Kh = R[0,1,1,0]/Det(gh)
Draw(Kh, param_mesh);

Thus the intrinsic computation gives an approximation to the same Gauss curvature value we found previously through the extrinsic Weingarten tensor.

## The Gauss-Bonnet theorem

Before we proceed to nonsmooth surfaces, it's useful to build more intuition using another remarkable  theorem that connects geometrical quantities (curvatures) to a topological one (Euler characteristic). Let $\cl S$ be a smooth surface $\cl S$ with boundary $\partial \cl S$, which may be empty. When it is not empty, let the boundary $\partial \cl S$ be smooth except at finitely many "vertices" $V$ collected into a set $\cl V_{\cl S}$. At each such vertex $V$, let $\alpha_V$ be the interior angle. The Gauss-Bonnet theorem asserts that 

\begin{align*}
\int_{\mathcal{S}}K\,ds + \int_{\partial \mathcal{S}}\kappa\,dl 
+ \sum_{V \in \cl V_{\cl S}}(\pi-\alpha_V)= 2\pi\chi.
\end{align*}

where, 
- $K$ is the Gauss curvature of $\cl S$, 
- $\kappa$ is the **geodesic curvature** of the smooth boundary curves, and 
- $\chi$ is the **Euler characteristic** of the surface.

### More on geodesic curvature $\kappa$

An extrinsic description of geodesic curvature of a curve on a surface is as follows. View the curve, perched atop the normal vector of the surface, like a bird looking down, as if it were a planar curve on a flat landscape, and then compute its planar  curvature to get $\kappa$. For example, geodesic curvatures of great circles of a sphere are zero because they will appear linear to the bird.

In fact the geodesic curvature of a curve in a Riemannian manifold is an intrinsic quantity computable from the metric alone: 

$$
\kappa = g(\nabla_{\tau} \tau, \mu)
$$

where $\tau, \mu$ is a $g$-orthonormal basis of the tangent space with $\tau$ tangent to the curve. NGSolve provides $\nabla_\tau \tau$ using the `specialcf.EdgeCurvature` function.


### Euler characteristic $\chi$

For a connected closed surface $\cl S$ with $\gamma$ holes, the Euler characteristic is 

$$
\chi = 2 -2 \gamma.
$$

For a surface $\cl S$ homeomorphic to a triangle, $\chi =1$.  More generally, given any triangulation of $\cl S$, the Euler characteristic equals  the number of triangles minus numer of edges plus number of vertices, i.e.,
$$
\chi=\# T-\# E + \# V.
$$

In [ ]:
def EulerChar(mesh):
    return mesh.nface - mesh.nedge + mesh.nv

### Examples

It is illustrative to apply the Gauss-Bonnet theorem for the following three examples:

<img width="60%" src="3sphereGBcases.png">

- The first has $\chi = 2$ and $K = 1/R^2$ where $R$ is the radius. The other two have $\chi = 1$.

- The second has a boundary whose geodesic curvature is $\kappa = 1/(R \tan \theta)$ where $\theta$ is the polar angle (complement of latitude) of the boundary curve.

- The third is a triangle all of whose vertex angles $\alpha_V$ are $\pi/2$.

**The hemisphere example again**

The hemisphere has no vertices. Its only boundary, being a great circle, has zero geodesic curvature. So Gauss-Bonnet  formula becomes 

$$
\int_{\cl S} K\; ds = 2 \pi.
$$

Both the intrinsic and extrinsic computation of $K$ yield approximations for which identity holds approximately.

In [ ]:
K = GaussCurvature()
intK = Integrate(K * ds, hemisph_mesh)
chiS = 2 * pi * EulerChar(hemisph_mesh)
print('Integral of K over hemisphere =', intK)
print('2 * pi * Euler characteristic =', chiS)   

In [ ]:
Kh = gh.Operator("Riemann")[0,1,1,0] / Det(gh)
intKh = Integrate(Kh * sqrt(Det(gh)), param_mesh)
chi = 2 * pi * EulerChar(param_mesh)
print('Integral of K over Rieman.mfd =', intKh)
print('2 * pi * Euler characteristic =', chi)   

## Nonsmooth cylindrical box

Consider a cylinder with its ends closed off. Then we obtain a cylindrical box, as shown below, which is not  a smooth surface.  

In [ ]:
cyl = Cylinder((0,0,0), X, r=2, h=3)
cylbox = Glue([cyl.faces[0], cyl.faces[1], cyl.faces[2]])
cylmesh = Mesh(OCCGeometry(cylbox).GenerateMesh(maxh=1)).Curve(4)
Draw(cylmesh, euler_angles=(4, 35, 10))     

In [ ]:
Draw(GaussCurvature(), cylmesh, euler_angles=(4, 35, 10));

We know that a cylindrical part of this closed surface (see [prior notebook](Extrinsic.ipynb)) has zero Gauss curvature. Of course the flat ends also have zero curvature. This explains why we got zero above.
It then appears that the Gauss Bonnet formula 

$$
\int_{\cl S} K = 2\pi \times 2
$$

does not hold in this case as the left hand side appears to evaluate to zero:

In [ ]:
Integrate(K * ds, cylmesh)  # LHS 

In [ ]:
2 * pi * EulerChar(cylmesh)  # RHS 

Of course, the surface is not smooth, so the theorem's statement is not contradicted by this example. Yet, this points to something that deserves more thought.

Indeed, by a small modification of the nonsmooth surface, we can get a smooth surface, for which the theorem must hold.

In [ ]:
cylsmooth = cyl.MakeFillet(cyl.edges, 0.5).faces
cylsmoothmesh = Mesh(OCCGeometry(cylsmooth).GenerateMesh(maxh=1)).Curve(5)
Draw(cylsmoothmesh);

In [ ]:
Integrate(GaussCurvature() * ds, cylsmoothmesh)  # Gauss Bonnet does hold

In [ ]:
Draw(GaussCurvature(), cylsmoothmesh);

The curvature has now become concentrated in the regions where the nonsmooth edge was smoothed out. As the smoothing region becomes tinier,  and the smooth surface becomes   closer to the nonsmooth one, the localized curvature on these regions  approaches an edge-delta-like-distribution.

We can even make a guess on what that edge curvature contribution should be. Since Gauss-Bonnet formula must hold separately for the smooth cylindrical face as well as the two flat circular disks, and since the geodesic curvature of the boundary of the former vanishes, the missing edge curvature contributions must be the geodesic curvature of the flat disks, which simply equals the reciprocal of their radius. Their integrals over the edges exactly equal the difference between $2 \pi \chi$ and the zero $\int_{\cl S} K$.


*This example leads us to suspect that the jump of the geodesic curvature across edges of a piecewise smooth manifold must be a source of curvature and should be included in the total curvature $\int_{\cl S} K\; ds$.*

## Nonsmooth cube

A cube has edges and corners that make it nonsmooth. If we were to apply the classical Gauss curvature formula in each smooth piece, we would get zero since all the facets of a cube are flat. Moreover, all of its edges are straight and are of zero geodesic curvature. So where does the curvature lie? 

Rounding out the edges and corners and then computing the Gauss curvature of the resulting smooth manifold give an answer.

In [ ]:
cube = Box( (0,0,0), (1,1,1) )
cubesmooth = cube.MakeFillet(cube.edges, 0.1)
cubesmoothmesh = Mesh(OCCGeometry(cubesmooth).GenerateMesh(maxh=0.2,perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(7)
Draw (GaussCurvature(), cubesmoothmesh);

This points to the vertices of the cube as sources of curvatures in a point-delta-like-distribution form. Indeed this is the classical *angle deficit* which Tullio Regge used to approximate curvature. Its definition follows. But first some geometric intuition. Rotate the cube and  focus on a vertex.

In [ ]:
Draw(cube, euler_angles=(50, 30, -10));

At the foreground vertex above,  note that the  sum of angles meeting at that vertex equals $3 \times \pi/2$, not $2\pi$ like on a flat surface. 

More generally, while the sum of angles made at a vertex by edges on a smooth manifold add up to $2 \pi$,  on a nonsmooth manifold, at a vertex such as a hill top,  the angles add up to less than $2 \pi$. This deficit of the angle sum is an indication of positive curvature.

<img width=40% src="angle_deficit.png"/>

The classical **angle deficit** at a vertex is defined by 

$$
\Theta_V = 2 \pi - \sum_T \sphericalangle_V^T.
$$

For the cube,  this angle deficit at each vertex equals  

$$
2 \pi - 3  \pi/2   = \pi/2.
$$

Since there are eight vertices, multiplying  by 8, we obtain $4 \pi$, which equal $2 \pi \chi$ in the Gauss Bonnet formula. 

*This example suggests that the total curvature $\int_{\cl S} K\; ds$ must include the sum of angle deficits at 
all vertices of a piecewise smooth manifold.*

## Generalized Gauss curvature for nonsmooth manifolds

The prior examples suggest  that a generalization of the notion of curvature on nonsmooth manifolds should  include curvature contributions on edges and vertices. We expect the former to take the form of an edge-delta of strength equal to the jump of geodesic curvature. We expect the latter to take the form of angle deficits at  vertices. This motivates the next definition.

Suppose $\cl S$ is a 2-manifold, subdivided into a triangulation $\cl T$, given a piecewise smooth Riemannian manifold structure via a metric $g$ in the Regge finite element space. Snce $g$ is smooth within each element $T$, we can view each $T$ as a Riemannian manifold by itself and compute its curvature $K_T$ by the classical formulas for the smooth case mentioned previously.  However, we must also account for sources of curvature at the element interfaces. We define  the **generalized Gauss curvature** of $\cl S$ (also known as the **distributional Gauss curvature**) as a functional acting on continuous scalar fields $u$  by 

$$
\widetilde K (u) = \sum_{V \in \mathcal{V}} \Theta_V \; u(V) +
\sum_{T \in \mathcal{T}} \left( \int_{(T, g)} K_T \, u 
+ \int_{ (\partial T, g) } \kappa \; u \right)
$$

for all $u$ in a Lagrange finite element space. Here at each vertex $V$ in the set of vertices $\cl V$ of the triangulation, the previously described angle deficit 
$\Theta_V $ acts as a curvature contribution. 

As in the case of the generalized Weingarten tensor, in order to visualize such a functional, we compute its  function representation obtained by lifting it into a finite element space. Namely, let  $\widetilde{K}_h$ be the unique function in the Lagrange space $V_h$ of degree $k+1$, when the metric is in the  Regge space of degree $k$, such that  

$$
      \sum_{T \in \cl T} \int_{(T, g)} \widetilde{K}_h \, w_h = \widetilde{K}(w_h)
$$

for all $w_h$ in $V_h$.



Note that each of the terms defining $\widetilde{K}(w_h)$ can be computed intrinsically, using only the metric $g$. Implementation of this intrisic calculation follows in the next cell. Of course, the terms can also be computed extrinsically if an embedding is known: this is pursued further down. 

### Intrinsic computation of generalized Gauss curvature

In intrinsic computation, we provide as input the 2D flat parameter domain meshed, a piecewise smooth Regge metric on it, and the degree of the Lagrange finite element space in which the generalized Gauss curvature is to be lifted into. No embedding information (and nothing 3D) is needed.

In [ ]:
def GeneralizedGaussCurvature(parameter_domain_mesh, gh, lift_order):
    
    t = ng.specialcf.tangential(2) # tangential vector at element edges
    n = -ng.specialcf.normal(2)    # normal vector at element edges
    bbnd_tang = ng.specialcf.VertexTangentialVectors(2) # tangent on edges into a vertex
    vt1 = bbnd_tang[:, 0]
    vt2 = bbnd_tang[:, 1]

    angldeficit = acos(vt1*vt2 / sqrt(vt1*vt1) / sqrt(vt2*vt2)) - \
                  acos(gh*vt1*vt2 / sqrt(gh*vt1*vt1) / sqrt(gh*vt2*vt2))
    edgecurv = ng.specialcf.EdgeCurvature(2)  # nabla_t t
    geodesicurv = sqrt(Det(gh)) / (t * (gh * t)) * \
                  (edgecurv * n + gh.Operator("christoffel2")[t, t, n])
    elementcurv= 1 / sqrt(Det(gh)) * gh.Operator("Riemann")[0, 1, 1, 0]

    V = ng.H1(parameter_domain_mesh, order=lift_order, dirichlet=".*")
    u, v = V.TnT()

    f = LinearForm(V)
    f += v * angldeficit * dx(element_vb=ng.BBND)
    f += v * geodesicurv * dx(element_boundary=True)
    f += v * elementcurv * dx

    liftK = GridFunction(V)     # lifted Gauss curvature
    with ng.TaskManager():
        M = BilinearForm(sqrt(Det(gh)) * u * v * dx)
        M.Assemble()
        f.Assemble()
        liftK.vec.data = M.mat.Inverse(V.FreeDofs()) * f.vec
        
    return liftK

### Extrinsic computation of generalized Gauss curvature

The same formula for generalized Gauss curvature can also be implemented in the extrinsic approach. Here, the metric on the surface is inherited from the ambient 3D Euclidean metric, so there is no need to provide a metric as input. We provide the entire (meshed) surface in 3D as input.  The degree of the Lagrange finite element space in which the generalized Gauss curvature is to be lifted into should also be provided as input.

In [ ]:
def GeneralizedGaussCurvatureExtrinsic(surface_mesh, lift_order):

    mu = Cross(ng.specialcf.normal(3), ng.specialcf.tangential(3))
    edgecurve = ng.specialcf.EdgeCurvature(3)  # nabla_t t
    bbndtang  = ng.specialcf.VertexTangentialVectors(3)
    vt1 = bbndtang[:,0] 
    vt2 = bbndtang[:,1] 
    V = H1(surface_mesh, order=lift_order)
    u,v = V.TnT()
    f = LinearForm(V)
    f += GaussCurvature()*v*ds(bonus_intorder=12)
    f += -mu * edgecurve*v*ds(element_boundary=True, bonus_intorder=12)
    f += -v*acos(vt1*vt2)*ds(element_vb=ng.BBND)

    with ng.TaskManager():
        f.Assemble()
        for i in range(f.space.mesh.nv):
            f.vec[i] += 2*pi

        Kh = GridFunction(V)
        mass = BilinearForm(u*v*ds(bonus_intorder=6)).Assemble().mat
        Kh.vec.data = mass.Inverse()*f.vec

    return Kh

## Example: Riemannian manifold approximated by piecewise flat metrics  

In this example, a smooth metric on a square parameter domain is approximated by piecewise flat metrics (in the lowest order Regge finite element space). The above-defined generalized Gauss curvature lifting, applied to the piecewise smooth metric approximations, then appears to converge to the exact curvature of the original smooth manifold as seen from the computations below. (Make the  mesh finer to visualize the convergence.)  

Note that the metric tensor of this example is induced by the embedding $\theta(x,y)= (x,y,f(x,y))$ of the graph of $f(x,y) = \frac{1}{2}(x^2+y^2)-\frac{1}{12}(x^4+y^4).$ This information is irrelevant to the computation below, but we mention it since it allows us to compute the exact metric and the exact curvature for reference.

In [ ]:
mapping = lambda x, y: (-1 + 2 * x, -1 + 2 * y)
mesh = MakeStructured2DMesh(quads=False, nx=8, ny=8, mapping=mapping)

# Exact metric:
gexact = CF((1 + (x - 1 / 3 * x**3) ** 2, (x - 1 / 3 * x**3) * (y - 1 / 3 * y**3),
             (x - 1 / 3 * x**3) * (y - 1 / 3 * y**3), 1 + (y - 1 / 3 * y**3) ** 2),
            dims=(2, 2))

# Exact Gauss curvature:
Kexact = 81 * (1 - x**2) * (1 - y**2) / (9 + x**2 * (x**2 - 3) ** 2 + y**2 * (y**2 - 3) ** 2) ** 2

# Exact curvature for reference:
Draw(Kexact, mesh, "K", order=3, deformation=True);

In [ ]:
# Approximate the metric by a piecewise constant (flat) metric gh
order = 0
Rg = ng.HCurlCurl(mesh, order=order)
gh = GridFunction(Rg)
gh.Set(gexact, dual=True, bonus_intorder=5)

# Use the intrinsic method to compute generalized Gauss curvature
Kh = GeneralizedGaussCurvature(mesh, gh, lift_order=order+1)
Draw(Kh, mesh, min=0, max=1, deformation=True);

## Example: Intersecting tori

In this example we consider a closed 3D surface of nontrivial genus. Knowing the surface in 3D, it is now convenient to use the extrinsic method to calculate its generalized Gauss curvature. The surface consists of one torus intersecting another, as made below.

In [ ]:
from netgen.occ import WorkPlane, Revolve, Translation, Axis

circ = WorkPlane(Axes((3, 0, 0), -Y, X)).Circle(1).Face()
torus = Revolve(circ, Axis((0, 0, 0), (0, 0, 1)), 360)
torus.faces.name = "torus"

torus2 = Translation((6.5, 0, 0))(torus)
torus2.faces.name = "torus2"
two_torus = Glue((torus2 - torus).faces["torus2"] +
                 (torus - torus2).faces["torus"])
torimesh = Mesh(OCCGeometry(two_torus).GenerateMesh(maxh=1)).Curve(5)
Draw(torimesh);

Appealing to the previously defined extrinsic method, we obtain a visualization of lifted generalized Gauss curvature that shows sharp variations at the nonsmooth intersection. 

In [ ]:
Kh = GeneralizedGaussCurvatureExtrinsic(torimesh, lift_order=6)
Draw(Kh, torimesh, max=0.3, min=-0.3);

Note that by construction, the lifted generalized Gauss curvature $\widetilde{K}_h$ on closed surfaces satisfies the Gauss-Bonnet formula, i.e., 

$$
\int_{\cl S} \widetilde{K}_h \; ds = 2\pi \chi.
$$

One can verify that this is true for the $\widetilde{K}_h$ we just computed, but not true if $\widetilde{K}_h$ is replaced by the classical Gauss curvature of the torus pieces: the outputs of the next two cells representing the left and right hand sides above are equal, while the output of the next third cell (integral of the classical Gauss curvature) is completely different since it ignores the curvature source at the nonsmooth interface.

In [ ]:
Integrate(Kh, torimesh.Boundaries(".*"), order=15)

In [ ]:
2 * pi * EulerChar(torimesh)

In [ ]:
Integrate(GaussCurvature(), torimesh.Boundaries(".*"), order=15)

## References


[<a href="https://doi.org/10.5802/smai-jcm.98">Gopalakrishnan, Neunteufel, Schöberl, Wardetzky:  Analysis of curvature approximations via covariant incompatibility and curl for Regge metrics, <i>The SMAI Journal of computational mathematics</i> (2023).</a>]

[<a href="https://arxiv.org/abs/2311.01603">Gopalakrishnan,  Neunteufel, Schöberl, Wardetzky: 
Generalizing Riemann curvature to Regge metrics (2025).</a>]

